# Figure 1

Start by loading some boiler plate: matplotlib, numpy, scipy, json, functools, and a convenience class.

In [1]:
%matplotlib inline
import matplotlib
matplotlib.rcParams['figure.figsize'] = (10.0, 16.0)
import matplotlib.pyplot as plt
import numpy as np
from scipy.interpolate import interp1d, InterpolatedUnivariateSpline
from scipy.optimize import bisect
import json
from functools import partial
class Foo: pass

And some more specialized dependencies:
 1. ``Slict`` provides a convenient slice-able dictionary interface
 2. ``Chest`` is an out-of-core dictionary that we'll hook directly to a globus remote using...
 3. ``glopen`` is an open-like context manager for remote globus files

In [2]:
from chest import Chest
from slict import CachedSlict
from glopen import glopen, glopen_many

Configuration for this figure.

In [3]:
config = Foo()
config.name     = "/home/maxhutch/tmp/Schmidt-Projection/Nu01D01/Nu01D01"
config.arch_end = "maxhutch#finarfin"
config.frame = 1
config.lower = .25
config.upper = .75

Open a chest located on a remote globus endpoint and load a remote json configuration file.

In [4]:
c = Chest(path      = "{:s}-results".format(config.name),
          open      = partial(glopen,      endpoint=config.arch_end),
          open_many = partial(glopen_many, endpoint=config.arch_end))
sc = CachedSlict(c)
with glopen(
            "{:s}.json".format(config.name), mode='r',
            endpoint = config.arch_end,
            ) as f:
    params = json.load(f)


We want to grab all the data for the selected frame.

In [12]:
T = sc[:,'frame'].keys()[config.frame]
frame = sc[T,:]
print(c._keys)
c.prefetch(frame.full_keys())

{(34.00335975933, 'TMax'): '/projects/alpha-nek/Schmidt-Projection/Nu01D01/Nu01D01-results/_b3dca67d5951fc365dcd88f0ae4add4b/TMax', (17.0083632295, 'TMax'): '/projects/alpha-nek/Schmidt-Projection/Nu01D01/Nu01D01-results/_f4241939a55695d37bf73a96c1160ea9/TMax', (6.027556073851, 'dx_max'): '/projects/alpha-nek/Schmidt-Projection/Nu01D01/Nu01D01-results/_2512f10a4e11991cea895485402fe75c/dx_max', (17.0083632295, 'd_yz'): '/projects/alpha-nek/Schmidt-Projection/Nu01D01/Nu01D01-results/_f4241939a55695d37bf73a96c1160ea9/d_yz', (28.0007829637, 't_xy'): '/projects/alpha-nek/Schmidt-Projection/Nu01D01/Nu01D01-results/_65bc78048f323c014fffca35cc72f59d/t_xy', (19.00405600516, 'u_xy'): '/projects/alpha-nek/Schmidt-Projection/Nu01D01/Nu01D01-results/_b58b992dd088d411c8b2f85040f6e180/u_xy', (42.00054280267, 'TMax'): '/projects/alpha-nek/Schmidt-Projection/Nu01D01/Nu01D01-results/_ae5a719a92a3e77868ee215e44dc9f02/TMax', (25.00598088275, 'u2_proj_z'): '/projects/alpha-nek/Schmidt-Projection/Nu01D01/Nu

FileNotFoundError: [Errno 2] No such file or directory: '/projects/alpha-nek/Schmidt-Projection/Nu01D01/Nu01D01-results/_777d45bbbcdf50d49c42c70ad7acf5fe/p_yz'

Plot the bubble height, the 'H' keys, vs. time.
Use a spline to compute the derivative.

In [ ]:
fig = plt.figure()

nx = frame['t_yz'].shape[0]
ny = frame['t_yz'].shape[1]

plt.imshow(frame['fz_yz'][:,int(ny*config.lower):int(ny*config.upper)].transpose(),
           vmin = -params['atwood']*params['g']*2,
           vmax =  params['atwood']*params['g']*2,
           origin='lower')
plt.colorbar();
plt.contour(frame['fz_yz'][:,int(ny*config.lower):int(ny*config.upper)].transpose(), 
            colors='k', 
            levels=[-params['atwood']*params['g'], params['atwood']*params['g']])
#axs[1].imshow(frame['fz_yz'].transpose(), origin='lower')

In [ ]:
def laplace(grid):
    from numpy.fft import fftn, ifftn, fftfreq
    kx = fftfreq(grid.shape[0])
    ky = fftfreq(grid.shape[1])
    kgrid = fftn(grid)
    for i in range(kgrid.shape[0]):
        for j in range(kgrid.shape[1]):
            if kx[i] != 0 or ky[j] != 0:
                kgrid[i,j] = kgrid[i,j] / (kx[i]**2 + ky[j]**2)
    kgrid[0,0] = 0.
    rgrid = np.array(ifftn(kgrid), dtype=float)
    return rgrid

In [ ]:
fig = plt.figure()

nx = frame['t_yz'].shape[0]
ny = frame['t_yz'].shape[1]

plt.imshow(frame['vorticity_yz'][:,int(ny*config.lower):int(ny*config.upper)].transpose(),
           origin='lower')
#axs[1].imshow(frame['fz_yz'].transpose(), origin='lower')
plt.colorbar();

In [ ]:
streamfunction = laplace(frame['vorticity_yz'][:,:]);

In [ ]:
fig = plt.figure()

nx = frame['t_yz'].shape[0]
ny = frame['t_yz'].shape[1]
print(np.max(streamfunction), np.min(streamfunction))

#background = np.tile(frame['t_yz'][:,int(ny*config.lower):int(ny*config.upper)].transpose(), (1,4))
background = np.tile(streamfunction[:,int(ny*config.lower):int(ny*config.upper)].transpose(), (1,4))
foreground = np.tile(streamfunction[:,int(ny*config.lower):int(ny*config.upper)].transpose(), (1,4))

plt.imshow(background,
           origin='lower', alpha=0.1)
plt.colorbar();
plt.contour(foreground,
           origin='lower', colors='k', levels=[-340337.480514*.6,0,.6*340337.389795])
#axs[1].imshow(frame['fz_yz'].transpose(), origin='lower')


In [ ]:
%install_ext http://raw.github.com/jrjohansson/version_information/master/version_information.py 
%load_ext version_information 
%version_information numpy, matplotlib, slict, chest, glopen, globussh